In [1]:
import getpass
import os
import numpy as np
import os
from dotenv import load_dotenv
from crypto_tools import *
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langgraph.prebuilt import create_react_agent
from langchain.agents import AgentExecutor, create_tool_calling_agent, Tool
from typing_extensions import TypedDict



In [2]:
from typing import Annotated, List

from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool



@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'\n{doc.page_content}\n'
            for doc in docs
        ]
    )


#Define Tools
tools = [
    Tool(name="Find_Similar_Plan", func=find_similar_plan, description="Finds the most similar plan from common plans based on the user's query."),
    Tool(name="Get_ETH_Balance", func=get_eth_balance, description="Fetch the ETH balance of a wallet."),
    Tool(name="Get_Token_Balance", func=get_token_balance, description="Fetch the balance of a specific token."),
    Tool(name="Lookup_Token_Address", func=lookup_token_address, description="Look up the contract address for a given token name."),
    Tool(name="Perform_Token_Swap", func=perform_token_swap, description="Perform a token swap from one asset to another."),
    Tool(name="Get_Transaction_Status", func=get_transaction_status, description="Fetch the status of a blockchain transaction."),
    Tool(name="Get_Recent_Transactions", func=get_recent_transactions, description="Fetch recent transactions from the latest block."),
    Tool(name="Estimate_Gas_Fee", func=estimate_gas_fee, description="Estimate current gas fees on the network."),
    Tool(name="Get_Best_Swap_Rate", func=get_best_swap_rate, description="Fetch the best swap rate from DEX aggregators."),
    Tool(name="Get_NFT_Collection", func=get_nft_collection, description="Fetch a wallet's NFT collection."),
    Tool(name="Get_NFT_Value", func=get_nft_value, description="Get the estimated value of an NFT."),
    Tool(name="Get_Token_Price", func=get_token_price, description="Fetch the current price of a token from market data."),
    Tool(name="Get_Historical_Prices", func=get_historical_prices, description="Retrieve historical price data for a token."),
    Tool(name="Get_Staking_Opportunities", func=get_staking_opportunities, description="Fetch available staking opportunities for a token."),
    Tool(name="Calculate_Staking_Yield", func=calculate_staking_yield, description="Calculate the staking yield for a given token and amount."),
    Tool(name="Create_Wallet", func=create_wallet, description="Create a new blockchain wallet."),
    Tool(name="Call_Smart_Contract_Function", func=call_smart_contract_function, description="Call a function on a smart contract."),
    Tool(name="Get_Contract_Data", func=get_contract_data, description="Fetch data from a smart contract."),
    Tool(name="Bridge_Tokens", func=bridge_tokens, description="Bridge tokens across chains."),
    Tool(name="Vote_on_Proposal", func=vote_on_proposal, description="Vote on a governance proposal."),
    Tool(name="Get_Governance_Tokens", func=get_governance_tokens, description="Fetch governance tokens held by a wallet."),
    Tool(name="Get_Tokens_From_Etherscan", func=get_tokens_from_etherscan, description="Fetch the list of tokens in a wallet using Etherscan."),
    Tool(name="Get_NFTs_From_Etherscan", func=get_nfts_from_etherscan, description="Fetch the list of NFTs in a wallet using Etherscan."),
]


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
tools[0]

Tool(name='Find_Similar_Plan', description="Finds the most similar plan from common plans based on the user's query.", func=<function find_similar_plan at 0x74bf9170a340>)

In [4]:
from langchain_core.messages import HumanMessage


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

members = ["Planner", "Executer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members


class routeResponse(BaseModel):
    next: Literal[*options]


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))


llm = ChatOpenAI(model="gpt-4o")


def supervisor_agent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

In [6]:
import functools
import operator
from typing import Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


planner_agent = create_react_agent(llm, tools=tools)
planner_node = functools.partial(agent_node, agent=planner_agent, name="Planner")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
executer_agent = create_react_agent(llm, tools=tools)
executer_node = functools.partial(agent_node, agent=executer_agent, name="Executer")

workflow = StateGraph(AgentState)
workflow.add_node("Planner", planner_node)
workflow.add_node("Executer", executer_node)
workflow.add_node("supervisor", supervisor_agent)

In [7]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

In [9]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="what is the wallet balance at 0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Planner'}}
----
{'Planner': {'messages': [HumanMessage(content='The wallet at address `0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97` has the following balances:\n\n### ETH Balance\n- **ETH:** 14.3838 ETH\n\n### Token Balances\n- **AITWTZ:** 0.0000\n- **USA:** 500.0000\n- **USDT:** 400.0000\n- **BTC:** 200.0000\n- **ETH:** 1.8724\n- **OCC:** 658.0000\n- **RealMcCoy:** 0.0000\n- **USDT:** 3,000,000.0000\n- **ETH:** 0.0373\n- **XAEA51:** 0.0000\n- **BTAI:** 0.0000\n- **Visit https://get-usdc.com to claim rewards:** 1049.0000\n- **ETH:** 1.0254\n- **ETH:** 0.8556\n- **ETH:** 2.5369\n- **ETH:** 1.1840\n- **ETH:** 0.9629\n- **ETH:** 0.9629\n- **ETH:** 1.3223\n- **Visit https://usdswap.org to claim rewards:** 3999.9900\n- **ЕTH:** 0.0442\n- **ΕTH:** 0.1155\n- **MAIL:** 48,999,999.0200\n- **Visit https://usd-coin.net to claim rewards:** 3999.9900\n- **SH三P三:** 38,769,610,461,393.6484\n- **HQG:** 1.7283\n- **ETH:** 200.0000\n- **NPQ:** 25.3643\n- **$ sUSD:** 826.1500\n- *